In [81]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos


In [82]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_a1_WALLET = WalletConfig("a1", "pin")

TRADER_a2_WALLET = WalletConfig("a2", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TRADER_3_WALLET = WalletConfig("T3", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_2_WALLET, TRADER_3_WALLET, TRADER_a1_WALLET , TRADER_a2_WALLET, TERMINATE_WALLET]


In [83]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/t8/z7v3pdr57d1_rrxyb2skvdtm0000gn/T/tmpu2th_mie
INFO:vega_sim.null_service:Launching GraphQL node at port 63274
INFO:vega_sim.null_service:Launching Console at port 63281
Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [84]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

In [85]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

tdai_id = vega.find_asset_id(symbol="tDAI")

mint_amount = {MM_WALLET.name: 10e12, TRADER_a1_WALLET.name: 10e12, TRADER_a2_WALLET.name: 10e12, TRADER_2_WALLET.name: 2000, TRADER_3_WALLET.name: 90, TERMINATE_WALLET.name: 2000}

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=mint_amount[MM_WALLET.name]
)
vega.mint(
    TRADER_a1_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a1_WALLET.name]
)
vega.mint(
    TRADER_a2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a2_WALLET.name]
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_2_WALLET.name]
)
vega.mint(
    TRADER_3_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_3_WALLET.name]
)
vega.mint(
    TERMINATE_WALLET.name,
    tdai_id,
    amount=mint_amount[TERMINATE_WALLET.name]
)

vega.wait_for_total_catchup()


In [86]:
vega.update_network_parameter(
            MM_WALLET.name, 
            parameter="market.margin.scalingFactors", 
            new_value="{\"search_level\": 1.5, \"initial_margin\": 2, \"collateral_release\": 3}"
        )

In [87]:
# Create a market
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=0,
        market_decimals=0,
        risk_aversion=0.000001,
        tau=0.1,
        sigma=1,
        
)

market_id = vega.all_markets()[0].id

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [88]:
# submit liquidity
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=100000,
    fee=0.001,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=55,
    delta_sell=55,
    is_amendment=False,
)

vega.forward('1s')
vega.forward('1s')

In [89]:
# Post GTC limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=5,
    price=5,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=1000,
    wait=True,
)

buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=10,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=10,
    wait=True,
)


In [90]:
# Check order status/ market state

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))
    print(vega.margin_levels(wallet.name))
    print(general+margin+bond - mint_amount[wallet.name])
    print(vega.order_status(order_id=buy_order_id))

print(vega.market_data(market_id=market_id))

Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.market_data. Be wary if prices/positions are not converted from int form


mm General = 9999999259417.434
mm Margin  = 640582.5664
mm Bond    = 100000.0
[MarginLevels(maintenance_margin=320291.2832, search_level=480436.9248, initial_margin=640582.5664, collateral_release_level=960873.8496, party_id='10b2deb312de88f7b846b9c4a7976c628b0fdfc9eb49ab557a73f04f46a9b728', market_id='c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f', asset='afc324717937c709ebb0a8b8c58d10d341ce43bdef35130d5dbe9efda8a9d6b4', timestamp=1638185465259066000)]
0.0
id: "3050c476dc1f3a0c2121862bf03bc64ef480f3918401f4531a6050ba7f01d75a"
market_id: "c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f"
party_id: "fb12b1c567a5b74f38d5c30444579895c62d27b07e0d126d9b36e6c25af78f75"
side: SIDE_BUY
price: "10"
size: 10
time_in_force: TIME_IN_FORCE_GTC
type: TYPE_LIMIT
created_at: 1638185457259066000
status: STATUS_FILLED
reference: "fb12b1c567a5b74f38d5c30444579895c62d27b07e0d126d9b36e6c25af78f75-e5e3466f-8972-4725-92c9-3b27cb3a467e"
updated_at: 1638185465259066000
versio

In [91]:
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=40,
    price=50,
    wait=True,
)
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(vega.margin_levels(wallet.name))
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))



[MarginLevels(maintenance_margin=7113.80719, search_level=10670.71078, initial_margin=14227.61438, collateral_release_level=21341.42157, party_id='10b2deb312de88f7b846b9c4a7976c628b0fdfc9eb49ab557a73f04f46a9b728', market_id='c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f', asset='afc324717937c709ebb0a8b8c58d10d341ce43bdef35130d5dbe9efda8a9d6b4', timestamp=1638185467259066000)]
mm General = 9999999885772.385
mm Margin  = 14227.61438
mm Bond    = 0.0
[MarginLevels(maintenance_margin=320.29129, search_level=480.43693, initial_margin=640.58258, collateral_release_level=960.87387, party_id='6840f87bd7e79bdfe6bbba774f07299cc7416b2ea9309c345545295b23ee054c', market_id='c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f', asset='afc324717937c709ebb0a8b8c58d10d341ce43bdef35130d5dbe9efda8a9d6b4', timestamp=1638185467259066000)]
T2 General = 1359.41742
T2 Margin  = 640.58258
T2 Bond    = 0
[]
T3 General = 90.0
T3 Margin  = 0
T3 Bond    = 0
[MarginLevels(maintenance

In [92]:

buy_order_id = vega.submit_order(
    trading_wallet=TRADER_3_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=100,
    wait=True,
)

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(vega.margin_levels(wallet.name))
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

[MarginLevels(maintenance_margin=0.0, search_level=0.0, initial_margin=0.0, collateral_release_level=14227.61438, party_id='10b2deb312de88f7b846b9c4a7976c628b0fdfc9eb49ab557a73f04f46a9b728', market_id='c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f', asset='afc324717937c709ebb0a8b8c58d10d341ce43bdef35130d5dbe9efda8a9d6b4', timestamp=1638185470259066000)]
mm General = 9999999885772.385
mm Margin  = 14227.61438
mm Bond    = 0.0
[MarginLevels(maintenance_margin=1921.74771, search_level=2882.62156, initial_margin=3843.49542, collateral_release_level=6405.82571, party_id='6840f87bd7e79bdfe6bbba774f07299cc7416b2ea9309c345545295b23ee054c', market_id='c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f', asset='afc324717937c709ebb0a8b8c58d10d341ce43bdef35130d5dbe9efda8a9d6b4', timestamp=1638185470259066000)]
T2 General = 0.0
T2 Margin  = 2000.0
T2 Bond    = 0
[MarginLevels(maintenance_margin=88.08012, search_level=132.12017, initial_margin=176.16024, collateral_r

In [93]:
# Check order status/ market state after amend
print(vega.order_status(order_id=buy_order_id))

print(vega.market_data(market_id=market_id))

Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form
Using function with raw data from data-node VegaService.market_data. Be wary if prices/positions are not converted from int form


id: "50ebbd1781462003d8dc60f2323adba9302b37f2666e261a0eb0ddce46b6fb19"
market_id: "c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f"
party_id: "c64375afba1098c85d14751e3411fef7dfa309a15b2c648f411389962bd7e741"
side: SIDE_BUY
price: "100"
size: 1
remaining: 1
time_in_force: TIME_IN_FORCE_GTC
type: TYPE_LIMIT
created_at: 1638185470259066000
status: STATUS_ACTIVE
reference: "c64375afba1098c85d14751e3411fef7dfa309a15b2c648f411389962bd7e741-70007b0b-c266-4c4b-9a01-0a5d7b40228a"
version: 1
batch_id: 1

mark_price: "10"
best_bid_price: "100"
best_bid_volume: 1
best_offer_price: "1000"
best_offer_volume: 10
best_static_bid_price: "100"
best_static_bid_volume: 1
best_static_offer_price: "1000"
best_static_offer_volume: 10
mid_price: "550"
static_mid_price: "550"
market: "c9ce643f3563cc177f42af95e85af4448e288cba6d4e980d0e519c6c67fd999f"
timestamp: 1638185473259066000
open_interest: 10
auction_end: 1638185472259066000
auction_start: 1638185468259066000
indicative_price: "0"
market